In [2]:
import numpy as np
import pandas as pd 

class desafioBiodiversidade():

    def __init__(self,filePath):
        try:
            self.f = open(filePath,"r")
            data = f.readlines()
            self.data = list(map(lambda line: line.split(";"), data))
            self.f.close()
        except:
            print("Error: cannot open file.")
            data = None  
    
    #1
    #Para cada coluna identique a quantidade de linhas com dados faltantes (em alguns casos, 
    #o dado faltante é uma string vazia, em outros casos é uma string contendo algum valor do 
    #tipo: "sem informação"). 
    #Faça um método que retorna a média de dados faltantes por coluna
    def retornaMediaDadosFaltantesPorColuna(self):
        data = self.data
        nlines = len(data)-1
        
        # lista de tamanho igual ao numero de colunas
        nblank = [0]*len(data[0])
        
        # cada posicao da lista armazena o contador de linhas faltantes da coluna correspondente
        for line in data:
            for ind,column in enumerate(line):
                if column == "Sem Informações" or column == "":
                    nblank[ind]+=1
        
        # calcula a media dos dados faltantes de cada coluna
        nblank = [np.round((x/nlines)*100,2) for x in nblank]
        
        # passa a lista para um dicionario
        # para impressao utilizando DataFrame do Pandas
        dict = {'Titulo': data[0], 'Value': nblank}
        df = pd.DataFrame(dict)
        print(df.to_string(index=False))
        
        
    #2
    #Para cada item identifique até qual nível taxônomico a ocorrência foi identificada.
    #Seis campos definem a espécie: Filo;Classe;Ordem;Familia;Genero;Especie do mais genérico 
    #(Filo) para o mais específico (Especie), é comum que em alguns casos, o preenchimento pare em 
    #família ou genêro por exemplo.
    #def retornaNivelTaxonomicoCadaOcorrencia():
    
    
    #3
    #Monte filtros de ocorrências por estados, nome de espécie (nome exato ou parte do nome) e 
    #categoria de ameaça, e outros filtros que julgar relevante.
    def retonaSeOcorrenciaExiste(self, filterAmeaca):
        self.AmeacaList = []
        self.dataAux = []
        self.dataT = []
        self.lines = [lines for lines in self.f]
        self.alldata = [dados.strip("\n").split(";") for dados in self.lines]
        # Lê cada coluna por vez        
        for j in range(len(self.alldata[0])):        
            for i in range(len(self.alldata)-1):
                self.dataAux.append(self.alldata[i][j])
                # Cria uma nova "matriz" transposta com os dados
            self.dataT.append(self.dataAux)
            self.dataAux = []
        
        
        ##### Filtros de busca
        # Filtro de ameaça
        for i in range(len(self.dataT)):
            if self.dataT[i][0] == "Estado de conservacao":
                for j in range(len(self.dataT[i])):
                    if self.dataT[i][j] == filterAmeaca :
                        self.AmeacaList.append(["Amostra {}: {}: ".format(j, self.dataT[i-12][j]), filterAmeaca])
                    
        return self.AmeacaList
        
        
    #4
    #Crie uma funcionalidade para avaliar se a informação de longitude e latitude 
    #corresponde a informação presente na localização, para isso você pode utilizar uma 
    #biblioteca de consulta reversa de lat/log como por exemplo o 
    #https://opencagedata.com/tutorials/geocode-in-python
    #def metodoPlotMap():
        

filePath = "portalbio_export_16-10-2019-14-39-54.csv"
biodiversidade = desafioBiodiversidade(filePath)
biodiversidade.retornaMediaDadosFaltantesPorColuna()
biodiversidade.retonaSeOcorrenciaExiste("Espécie Ameaçada")


Error: cannot open file.


[['Amostra 21: Elaenia ridleyana: ', 'Espécie Ameaçada'],
 ['Amostra 22: Puffinus lherminieri: ', 'Espécie Ameaçada'],
 ['Amostra 28: Fregata ariel: ', 'Espécie Ameaçada'],
 ['Amostra 29: Fregata minor: ', 'Espécie Ameaçada'],
 ['Amostra 30: Pterodroma arminjoniana: ', 'Espécie Ameaçada'],
 ['Amostra 31: Rudgea coronata: ', 'Espécie Ameaçada'],
 ['Amostra 37: Serjania: ', 'Espécie Ameaçada'],
 ['Amostra 38: Serjania: ', 'Espécie Ameaçada'],
 ['Amostra 39: Serjania: ', 'Espécie Ameaçada'],
 ['Amostra 40: Serjania: ', 'Espécie Ameaçada'],
 ['Amostra 41: Serjania: ', 'Espécie Ameaçada'],
 ['Amostra 42: Serjania: ', 'Espécie Ameaçada'],
 ['Amostra 43: Serjania: ', 'Espécie Ameaçada'],
 ['Amostra 45: Serjania: ', 'Espécie Ameaçada'],
 ['Amostra 49: Erythroxylum: ', 'Espécie Ameaçada'],
 ['Amostra 50: Eugenia: ', 'Espécie Ameaçada'],
 ['Amostra 51: Erythroxylum: ', 'Espécie Ameaçada'],
 ['Amostra 55: Psychotria: ', 'Espécie Ameaçada'],
 ['Amostra 58: Smilax: ', 'Espécie Ameaçada'],
 ['Amostr